In [1]:
import pandas as pd

In [2]:
with open("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt") as f:
    for line in f:
        print(line)
        break

{"task_id": "4a06139e", "domain": "UPDATE_CALENDAR", "bot_prompt": "Schedule the user's meeting request", "bot_role": "You are a bot designed to help schedule meetings on a calendar. ", "user_prompt": " You have a meeting saved for March 24th. Ask the chatbot to delete the meeting", "user_role": "You are interacting with a meeting scheduling bot"}



In [3]:
tasks = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt", lines=True)

In [4]:
dialog = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/ORDER_PIZZA.txt", lines=True)

In [23]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/*.txt"):
    txtfiles.append(file)

In [24]:
metalwoz = pd.read_json(txtfiles[0], lines=True)
for txt in txtfiles[1:]:
    tmp = pd.read_json(txt, lines=True)
    metalwoz = pd.concat([metalwoz,tmp])


In [46]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/*.json"):
    df = pd.read_json(file)
    break

In [58]:
a = "This is the first context"

In [154]:
schema = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/schema.json")
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/dev/schema.json")])
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/test/schema.json")])

In [126]:
slots = []
for idx, row in df.iterrows():
#     print(row)
    service = row['service_name']
    for i in row.slots:
        slots.append(service+"_"+i['name'])
#     for i in row.intents:
#         slots.extend([service+"_"+j for j in i['required_slots']])
#         slots.extend([service+"_"+j for j in list(i['optional_slots'].keys())])
    

In [167]:
schema[schema.service_name == "Restaurants_1"].intents.tolist()

[[{'name': 'ReserveRestaurant',
   'description': 'Reserve a table at a restaurant',
   'is_transactional': True,
   'required_slots': ['restaurant_name', 'city', 'time'],
   'optional_slots': {'date': '2019-03-01', 'party_size': '2'}},
  {'name': 'FindRestaurants',
   'description': 'Find a restaurant of a particular cuisine in a city',
   'is_transactional': False,
   'required_slots': ['cuisine', 'city'],
   'optional_slots': {'has_live_music': 'dontcare',
    'serves_alcohol': 'dontcare',
    'price_range': 'dontcare'}}]]

In [191]:
import glob

txtfiles = []
dialog = None
for i in ['train', 'dev', 'test']:
    for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/%s/*.json" % i):
        txtfiles.append(file)
        dialog = pd.concat([dialog, pd.read_json(file)])

In [172]:
frames = dialog.iloc[0]

In [173]:
dialog.iloc[0]

dialogue_id                                               100000
services                                         [Restaurants_1]
turns          [{'frames': [{'service': 'Restaurants_1', 'slo...
Name: 0, dtype: object

In [156]:
dialog.iloc[0]

dialogue_id                                               100000
services                                         [Restaurants_1]
turns          [{'frames': [{'service': 'Restaurants_1', 'slo...
Name: 0, dtype: object

In [254]:
prop2desc = {}
for idx, s in schema.iterrows():
    service = s.service_name
    for i in s.slots:
        prop2desc[service+"_"+i['name']] = i['description']


In [255]:
len(prop2desc), schema.service_name.nunique()

365

In [ ]:
train_data = []
qid = 1
for idx, row in dialog.iterrows():
    frames = row['turns']
    context = []
    
    if str(frames)=='nan':
        continue
        
    for f in frames:
        service = f['frames'][0]['service']
        if len(f['frames']) == 1:
            utterance = f['utterance']
            if len(f['frames'][0]['slots']) != 0:
                terms = []
                for s in f['frames'][0]['slots']:
                    _context = " ".join(context)
                    answer_start = len(_context) + s["start"] + 1
                    _context += " " +utterance.replace(utterance[s["start"]:s["exclusive_end"]], s['slot'].replace("_", " "))
                    _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
                                                          'question': prop2desc[service+"_"+s['slot']], 'answers':[{'text':s['slot'].replace("_"," "), 
                                                           'answer_start': answer_start}]}]}
                    train_data.append(_json)
                    qid += 1
            context.append(utterance)

In [ ]:
# Save as a JSON file
import json
with open('data/dialog/train.json', 'w') as f:
    json.dump(train_data, f)

In [414]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging
import torch

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('bert', 'bert-large-uncased-whole-word-masking-finetuned-squad', use_cuda=torch.cuda.is_available()
, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

# Train the model with JSON file
model.train_model('data/dialog/train.json')

# The list can also be used directly
# model.train_model(train_data)

# Evaluate the model. (Being lazy and evaluating on the train data itself)
result, text = model.eval_model('data/dialog/train.json')

print(result)
print(text)

print('-------------------')

# Making predictions using the model.
to_predict = [{'context': 'This is the context used for demonstrating predictions.', 'qas': [{'question': 'What is this context?', 'id': '0'}]}]

print(model.predict(to_predict))

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
  0%|          | 0/17 [00:00<?, ?it/s]INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000000
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 0
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answering_utils:tokens: [CLS] [UNK] or town where the attraction is located [SEP] [UNK] ' m looking for some free attractions to check out . [UNK] in particular ? [UNK] of a place of interest in location . [SEP]
INFO:simpletransformers.question_answering.question_answering_utils:token_to_orig_map: 10:0 11:0 12:0 13:1 14:2 15:3 16:4 17:5 18:6 19:7 20:8 21:8 22:9 23:10 24:11 25:11 26:12 27:13 28:14 29:15 30:16 31:17 32:18 33:19 34:19
INFO:simpletransformers.ques

INFO:simpletransformers.question_answering.question_answering_utils:segment_ids: 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:simpletransformers.question_answering.question_answering_utils:start_position: 82
INFO:simpletransformers.question_answering.question_answering_utils:end_position: 83
INFO:simpletransformers.question_answering.question_answering_utils:answer: hotel name
INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000003
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 3
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answeri

INFO:simpletransformers.question_answering.question_answering_utils:start_position: 100
INFO:simpletransformers.question_answering.question_answering_utils:end_position: 101
INFO:simpletransformers.question_answering.question_answering_utils:answer: hotel name
INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000005
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 5
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answering_utils:tokens: [CLS] [UNK] review rating for the hotel [SEP] [UNK] ' m looking for some free attractions to check out . [UNK] in particular ? [UNK] of a place of interest in [UNK] . [UNK] ' s a cool [UNK] [UNK] called [UNK] [UNK] . [UNK] it kid - friendly ? [UNK] ' s not the best place for kids . [UNK] right , thanks . [UNK] else [

INFO:simpletransformers.question_answering.question_answering_utils:segment_ids: 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:simpletransformers.question_answering.question_answering_utils:start_position: 118
INFO:simpletransformers.question_answering.question_answering_utils:end_position: 119
INFO:simpletransformers.question_answering.question_answering_utils:answer: hotel name
INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000007
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 7
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answe

INFO:simpletransformers.question_answering.question_answering_utils:token_is_max_context: 8:True 9:True 10:True 11:True 12:True 13:True 14:True 15:True 16:True 17:True 18:True 19:True 20:True 21:True 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:True 53:True 54:True 55:True 56:True 57:True 58:True 59:True 60:True 61:True 62:True 63:True 64:True 65:True 66:True 67:True 68:True 69:True 70:True 71:True 72:True 73:True 74:True 75:True 76:True 77:True 78:True 79:True 80:True 81:True 82:True 83:True 84:True 85:True 86:True 87:True 88:True 89:True 90:True 91:True 92:True 93:True 94:True 95:True 96:True 97:True 98:True 99:True 100:True 101:True 102:True 103:True 104:True 105:True 106:True 107:True 108:True 109:True 110:True 111:True 112:True 113:True 114:True 115:True 116:True 117:True 118:True 119:T

...

INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000023
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 15
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answering_utils:tokens: [CLS] [UNK] date for the hotel reservation [SEP] [UNK] ' m looking for some free attractions to check out . [UNK] in particular ? [UNK] of a place of interest in [UNK] . [UNK] ' s a cool [UNK] [UNK] called [UNK] [UNK] . [UNK] it kid - friendly ? [UNK] ' s not the best place for kids . [UNK] right , thanks . [UNK] else [UNK] can help with ? [UNK] ' ll be a 2 room hotel there . [UNK] about 1 [UNK] [UNK] [UNK] with its 4 . 5 rating ? [UNK] anything else ? [UNK] ' s 1 [UNK] [UNK] [UNK] with a 4 . 5 rating . [UNK] anything else ? [UNK] ' s 11 [UNK] with a 4 . 3 [SEP]
INFO:simp

INFO:simpletransformers.question_answering.question_answering_utils:end_position: 101
INFO:simpletransformers.question_answering.question_answering_utils:answer: check out date
INFO:simpletransformers.question_answering.question_answering_utils:*** Example ***
INFO:simpletransformers.question_answering.question_answering_utils:unique_id: 1000000025
INFO:simpletransformers.question_answering.question_answering_utils:example_index: 16
INFO:simpletransformers.question_answering.question_answering_utils:doc_span_index: 0
INFO:simpletransformers.question_answering.question_answering_utils:tokens: [CLS] [UNK] of the hotel [SEP] [UNK] ' m looking for some free attractions to check out . [UNK] in particular ? [UNK] of a place of interest in [UNK] . [UNK] ' s a cool [UNK] [UNK] called [UNK] [UNK] . [UNK] it kid - friendly ? [UNK] ' s not the best place for kids . [UNK] right , thanks . [UNK] else [UNK] can help with ? [UNK] ' ll be a 2 room hotel there . [UNK] about 1 [UNK] [UNK] [UNK] with its

INFO:simpletransformers.question_answering.question_answering_utils:segment_ids: 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
INFO:simpletransformers.question_answering.question_answering_utils:start_position: 109
INFO:simpletransformers.question_answering.question_answering_utils:end_position: 110
INFO:simpletransformers.question_answering.question_answering_utils:answer: street address
100%|██████████| 17/17 [00:00<00:00, 60.47it/s]


Running loss: 2.688654



KeyboardInterrupt: 